# Global Parameters & Imports for below functions:

In [ ]:
import requests
import html
import re
limit = 3 #no of stores to call
radius = 15 #radius in which to look

# Sainsbury's API:
## Documentation:
### Base URL:
https://stores.sainsburys.co.uk/api/v1/stores/
### Parameters:
* fields: Not sure what that means. Default value: <code>fields=slfe-list-2.21</code>
* api_client_id: Not sure either. Default value: <code>api_client_id=slfe</code>
* lat: Latitude of search area. Example: <code>lat=51.5073509</code>
* lon: Longitude of search area. Example: <code>lon=-0.1277583</code>
* limit: Stores displayed per page. Default value: <code>limit=25</code>
* store_type: Type of stores displayed. Can contain *main* for superstores and/or *local* for local stores. Example: <code>stores_type=main,local</code>
* sort: How results are sorted. Only found *by_distance* so far. Default value: <code>sort=by_distance</code>
* within: Max distance for which results should be shown. Not sure which unit of measurement is used (miles?). Seems to take a value between 0 and 500. Example: <code>within=15</code>
* page: Page of paginated results. Example: <code>page=1</code>

### Response:
NOTE: Day 0 is monday, and day 6 is Sunday.
Example response:
<pre>{'page_meta': {'limit': 25, 'offset': 0, 'total': 372},
 'results': [{'additional_data': {'main_store_branded_as_local': None,
  'restaurant_branded_as_fresh_kitchen': None},
 'closed': None,
 'code': '4286',
 'contact': {'address1': '36-37 Strand',
  'address2': '',
  'city': 'London',
  'country': 'England',
  'country_code': '',
  'country_name': '',
  'county': '',
  'fax_number': '',
  'manager': 'Debbie Farrow',
  'post_code': 'WC2N 5HY',
  'state': '',
  'telephone': '02079307006'},
 'distance': 0.17963147389184483,
 'district_code': '4286',
 'exception_times': [],
 'location': {'lat': '51.50888', 'lon': '-0.12438'},
 'name': 'Wsmtr The Strand Loc',
 'open': {'current_date': '2020-06-23',
  'current_time': '14:02',
  'is_open': True,
  'open_until': '21:00',
  'today': {'24hrs': False,
   'end_time': '21:00',
   'exception': False,
   'start_time': '07:00',
   'times': [{'end_time': '21:00', 'start_time': '07:00'}]},
  'tomorrow': {'24hrs': False,
   'end_time': '21:00',
   'exception': False,
   'start_time': '07:00',
   'times': [{'end_time': '21:00', 'start_time': '07:00'}]}},
 'opening_times': [{'day': 0,
   'end_time': '21:00',
   'start_time': '07:00',
   'times': [{'end_time': '21:00', 'start_time': '07:00'}]},
  {'day': 1,
   'end_time': '21:00',
   'start_time': '07:00',
   'times': [{'end_time': '21:00', 'start_time': '07:00'}]},
  {'day': 2,
   'end_time': '21:00',
   'start_time': '07:00',
   'times': [{'end_time': '21:00', 'start_time': '07:00'}]},
  {'day': 3,
   'end_time': '21:00',
   'start_time': '07:00',
   'times': [{'end_time': '21:00', 'start_time': '07:00'}]},
  {'day': 4,
   'end_time': '21:00',
   'start_time': '07:00',
   'times': [{'end_time': '21:00', 'start_time': '07:00'}]},
  {'day': 5,
   'end_time': '21:00',
   'start_time': '07:00',
   'times': [{'end_time': '21:00', 'start_time': '07:00'}]},
  {'day': 6,
   'end_time': '21:00',
   'start_time': '07:00',
   'times': [{'end_time': '21:00', 'start_time': '07:00'}]}],
 'opening_times_message': None,
 'originator_id': None,
 'other_name': 'Westminster The Strand Local',
 'store_type': 'local'}]
 ...
 } </pre>
 
# Code for retrieving Sainsbury's opening times data:


In [ ]:
def get_sainsburys_data(lat,lng):
    API_URL = "https://stores.sainsburys.co.uk/api/v1/stores/"
    params = {
        'fields': 'slfe-list-2.21',
        'api_client_id': 'slfe',
        'store_type': 'main,local',
        'sort': 'by_distance',
        'within': radius,
        'limit': limit,
        'page': '1',
        'lat': lat,
        'lon': lng,
    }
    rq = requests.get(API_URL, params=params)
    if rq.status_code != 200:
        # print(rq.json())
        return False
    res = rq.json()
    dayKeys = { 0 : 'Monday', 1 : 'Tuesday', 2 : 'Wednesday', 3 : 'Thursday', 4 : 'Friday', 5 : 'Saturday', 6 : 'Sunday'}
    i=0
    #INSERT POTENTIAL CHECK THAT STORE MATCHES DESIRED STORENAME!! (res[0]['name'] or res[0]['other_name'])
    #e.g.:
    # while True:
    #     if name != res[0]['name'] or res[0]['other_name']:
    #         i += 1
    #     else:
    #         break
    openingHours = res['results'][i]['opening_times']
    hoursArray = []
    for index in range(len(openingHours)):
        key = openingHours[index]['day']
        actualHours = []
        while len(hoursArray) < key:
            #Incase store is closed on a day of the week and therefore not included in the list
            closedDayHours = {  'day' : dayKeys[len(hoursArray)],
                                'open' : False  }
            hoursArray.append(closedDayHours)
        for timeSlotNumber in range(len(openingHours[index]['times'])):
            try:
                actualHoursDict = {}
                actualHoursDict['open'] = openingHours[index]['times'][timeSlotNumber]['start_time']
                actualHoursDict['close'] = openingHours[index]['times'][timeSlotNumber]['end_time']
                actualHours.append(actualHoursDict)
            except:
                #If dictionary keys start_time or end_time dont exist, assume store is closed on that day
                closedDayHours = {  'day' : dayKeys[key],
                                'open' : False  }
                hoursArray.append(closedDayHours)
                break
        keyHours = {'day' : dayKeys[key],
                'open' : True,
                'hours' : actualHours
                }
        hoursArray.append(keyHours)
    while len(hoursArray) < 7:
            #Incase store is closed on a day of the week (at the end of the list/week) and therefore not included in the list
            closedDayHours = {  'day' : dayKeys[len(hoursArray)],
                                'open' : False  }
            hoursArray.append(closedDayHours)
    return hoursArray

# ASDA's API
It is of note that ASDA's system returns a potentially extrememely large quanitity of different qualitative data regarding the store in question, although not all of this data is returned for each store's "response" field. Although, the full schema of possible information for the API to return can be found under "schema" of the API call.

Notes:
Has Click and Collect info.
Diesel price is potentially something that could be displayed (especially if this is displayed on stores like BP Garage etc..).
## Base URL:
https://storelocator.asda.com/index.html
## Parameters:
* q = appears to send off to an external (Google?) API in order to translate whatever form of address you input. Therefore latitude and longitude may be used as so: "latitude,longitude"
* qn = generally seems to mimic what is input into q when used through the website, however when "My Location" button is used, q = lat,lon and qn = "My Location"    **(optional)**
* l = language ("en" for english)  **(optional)**

## Headers:
* accept = "application/json" (otherwise will return html (?))

## Sample API Call:
https://storelocator.asda.com/index.html?q=52.635875,%201.306051&qp=52.635875,%201.306051&l=en
## Sample Response:


In [ ]:
baseUrl: ""
businessId: 2479739
errors: null
featureId: "Search"
isHttps: true
isPrimaryLocale: true
isStaging: false
locale: "en"
queryParams: {l: ["en"], q: ["52.635875, 1.306051"], qp: ["52.635875, 1.306051"]}
response: {count: 2, entities: [{derivedData: {address: {countryName: "United Kingdom"}, uber: {,…}},…},…],…}
    count: 2
    entities: [{derivedData: {address: {countryName: "United Kingdom"}, uber: {,…}},…},…]
        0: {derivedData: {address: {countryName: "United Kingdom"}, uber: {,…}},…}
            derivedData: {address: {countryName: "United Kingdom"}, uber: {,…}}
                address: {countryName: "United Kingdom"}
                    countryName: "United Kingdom"
                uber: {,…}
                    url: "https://m.uber.com/ul/?action=setPickup&client_id=KXQcwoj2Zb8ymDzKgVgbIaDE5iAE_TAj&dropoff%5Bformatted_address%5D=Hall%20Road%2C%20null%2C%20Norwich%2C%20null&dropoff%5Blatitude%5D=52.60555&dropoff%5Blongitude%5D=1.28634&dropoff%5Bnickname%5D=Asda%20Norwich%20Hall%20Road%20Superstore&pickup=my_location"
                    wrappedUrl: "http://a.gotoloc.com/uber/F3EkTqKGX2A"
            distance: {distanceKilometers: 3.4855743361404476, distanceMiles: 2.1658354808794438, id: "4383"}
                distanceKilometers: 3.4855743361404476
                distanceMiles: 2.1658354808794438
                id: "4383"
            listings: {,…}
                googleMyBusiness: {placeId: "ChIJKZs0vAHk2UcRkaLqm7RF3hk", url: "https://maps.google.com/maps?cid=1864003937790239377"}
                    placeId: "ChIJKZs0vAHk2UcRkaLqm7RF3hk"
                    url: "https://maps.google.com/maps?cid=1864003937790239377"
            profile: {,…}
                address: {city: "Norwich", countryCode: "GB", extraDescription: null, line1: "Hall Road", line2: null,…}
                    city: "Norwich"
                    countryCode: "GB"
                    extraDescription: null
                    line1: "Hall Road"
                    line2: null
                    line3: null
                    postalCode: "NR4 6DP"
                    region: null
                    sublocality: null
                c_24HourStore: false
                c_24HourStoreWithSundayClosure: false
                c_aboutSectionCTA: {text: "ASDA.com", url: "https://www.asda.com/"}
                    text: "ASDA.com"
                    url: "https://www.asda.com/"
                c_aboutSectionDescription: "Asda Norwich Hall Road is part of the Asda group; one of Britain’s leading grocery retailers. Asda offers great prices and quality products helping customers save money & live better. Visit us in-store and get groceries delivered to anywhere in the UK, or collect from your local store or petrol station at a convenient time for you."
                c_aboutSectionTitle: "About ASDA - Norwich Hall Road"
                c_alertBannerBackgroundColor: "#5e9b1a"
                c_alertBannerDatePosted: {day: 6, month: 5, year: 2020}
                    day: 6
                    month: 5
                    year: 2020
                c_alertBannerText: "We’re so grateful for the amazing work of our NHS workers, Care Home Workers and Carers, which is why we continue to offer priority access to our stores for these customers <b>between 8am and 9am each Monday, Wednesday and Friday morning—as well as an exclusive hour of browsing on Sunday morning before our stores open.</b>↵↵Please try and avoid coming to our stores at these times to let these vital workers get what they need, and we ask these customers to bring their work ID with them to help our colleagues spot you."
                c_alertBannerTitle: "<b>Priority Shopping</b>"
                c_appleName: "Asda Superstore"
                c_asdaLiving: false
                c_babyChanging: true
                c_bakery: true
                c_bakeryCounter: false
                c_barclaysBank: false
                c_butcherCounter: false
                c_cafe: true
                c_carPark: true
                c_carparkType: ["FREE"]
                    0: "FREE"
                c_cashMachine: true
                c_changingPlacesFacility: false
                c_clickAndCollect: true
                c_clickAndCollectCardTitle: "Click & Collect"
                c_clickAndCollectLockersText: "External Collection Point"
                c_clickAndCollectText: "Click to book your groceries collection at a time to suit you"
                c_communityChamionNominationText1: "How to nominate a group for Green Token Giving"
                c_communityChamionNominationText2: "Email or speak to your Community Champion in store to nominate a cause for Green token giving scheme."
                c_communityChampion: true
                c_communityChampionContactText: "For further information contact your community champion at Community_SouthNorwich@asda.co.uk"
                c_communityChampionNominationLink: "http://www.asda.com"
                c_communityChampionsCTA: {text: "Read More", url: "https://www.asda.com/community-champions"}
                    text: "Read More"
                    url: "https://www.asda.com/community-champions"
                c_communityChampionsImage: {image: {height: 119,…}}
                    image: {height: 119,…}
                        height: 119
                    thumbnails: [{height: 48, url: "//dynl.mktgcdn.com/p/_d5C6bX8hYCMiZGCbHmiVVqBslZRmeue99ETKUi91tI/196x48.png",…}]
                        0: {height: 48, url: "//dynl.mktgcdn.com/p/_d5C6bX8hYCMiZGCbHmiVVqBslZRmeue99ETKUi91tI/196x48.png",…}
                            height: 48
                            url: "//dynl.mktgcdn.com/p/_d5C6bX8hYCMiZGCbHmiVVqBslZRmeue99ETKUi91tI/196x48.png"
                            width: 196
                    url: "//dynl.mktgcdn.com/p/_d5C6bX8hYCMiZGCbHmiVVqBslZRmeue99ETKUi91tI/480x119.png"
                    width: 480
                c_communityChampionsText: {,…}
                    text(list): ["Local grants", "Green token giving", "Product donation", "Using Asda facilities", "Volunteering"]
                        0: "Local grants"
                        1: "Green token giving"
                        2: "Product donation"
                        3: "Using Asda facilities"
                        4: "Volunteering"
                    text(multi-line): "Are you a charity or not for profit organisation looking for support from Asda? Then contact your Local Community Champion to discuss:"
                c_communityRoom: false
                c_contactSectionTitle: "Contact"
                c_costaCoffee: true
                c_customerWC: true
                c_deliCounter: false
                c_directoryCounty: "East Of England"
                c_disabledFacilities: true
                c_division: "South"
                c_electricCarCharging: true
                c_electricVehicleChargingPoint: true
                c_electricalGoods: true
                c_ethnicFoods: false
                c_expressDiner: true
                c_expressPizzaCounter: false
                c_expressRotisserie: false
                c_facilitiesList: ["Car Park", "Cash Machine", "Customer WC", "Disabled Facilities", "Baby Changing",…]
                    0: "Car Park"
                    1: "Cash Machine"
                    2: "Customer WC"
                    3: "Disabled Facilities"
                    4: "Baby Changing"
                    5: "Electric Vehicle Charging Point"
                    6: "Helium Balloons"
                c_facilitiesTitle: "Facilities"
                c_featuredServiceCardCafe: {hours: {holidayHours: [],…}, photo: {image: {height: 768,…}},…}
                    hours: {holidayHours: [],…}
                        holidayHours: []
                        normalHours: [{day: "MONDAY", intervals: [], isClosed: true}, {day: "TUESDAY", intervals: [], isClosed: true},…]
                            0: {day: "MONDAY", intervals: [], isClosed: true}
                                day: "MONDAY"
                                intervals: []
                                isClosed: true
                            1: {day: "TUESDAY", intervals: [], isClosed: true}
                                day: "TUESDAY"
                                intervals: []
                                isClosed: true
                            2: {day: "WEDNESDAY", intervals: [], isClosed: true}
                                day: "WEDNESDAY"
                                intervals: []
                                isClosed: true
                            3: {day: "THURSDAY", intervals: [], isClosed: true}
                                day: "THURSDAY"
                                intervals: []
                                isClosed: true
                            4: {day: "FRIDAY", intervals: [], isClosed: true}
                                day: "FRIDAY"
                                intervals: []
                                isClosed: true
                            5: {day: "SATURDAY", intervals: [], isClosed: true}
                                day: "SATURDAY"
                                intervals: []
                                isClosed: true
                            6: {day: "SUNDAY", intervals: [], isClosed: true}
                                day: "SUNDAY"
                                intervals: []
                                isClosed: true
                photo: {image: {height: 768,…}}
                    image: {height: 768,…}
                        height: 768
                        sourceUrl: "http://a.mktgcdn.com/p/FKZfwMNvVM2oQ1EvQIPTq8dLEkWU1_sFy592GblCSAA/1152x768.png"
                        thumbnails: [,…]
                            0: {height: 412, url: "//dynl.mktgcdn.com/p/FKZfwMNvVM2oQ1EvQIPTq8dLEkWU1_sFy592GblCSAA/619x412.png",…}
                                height: 412
                                url: "//dynl.mktgcdn.com/p/FKZfwMNvVM2oQ1EvQIPTq8dLEkWU1_sFy592GblCSAA/619x412.png"
                                width: 619
                            1: {height: 400, url: "//dynl.mktgcdn.com/p/FKZfwMNvVM2oQ1EvQIPTq8dLEkWU1_sFy592GblCSAA/600x400.png",…}
                                height: 400
                                url: "//dynl.mktgcdn.com/p/FKZfwMNvVM2oQ1EvQIPTq8dLEkWU1_sFy592GblCSAA/600x400.png"
                                width: 600
                            2: {height: 130, url: "//dynl.mktgcdn.com/p/FKZfwMNvVM2oQ1EvQIPTq8dLEkWU1_sFy592GblCSAA/196x130.png",…}
                                height: 130
                                url: "//dynl.mktgcdn.com/p/FKZfwMNvVM2oQ1EvQIPTq8dLEkWU1_sFy592GblCSAA/196x130.png"
                                width: 196
                        url: "//dynl.mktgcdn.com/p/FKZfwMNvVM2oQ1EvQIPTq8dLEkWU1_sFy592GblCSAA/1152x768.png"
                        width: 1152
                    text(multi-line): "Our Asda Cafés have your dining experience covered! We’ve got great food at low prices - just what you’d expect from Asda. What’s more, kids Eat Free Monday-Friday after 3pm when an adult spends just £4, and Pensioners get 10% off all café items on a Wednesday."
                    text(single-line): "Cafe"
                c_firstChoiceTravel: false
                c_fishCounter: false
                c_foodToGo: true
                c_format: "Superstore"
                c_fuelPrice: {diesel: "112.70", effectiveDate: {day: 10, month: 7, year: 2020}, unleaded: "107.70"}
                    diesel: "112.70"
                    effectiveDate: {day: 10, month: 7, year: 2020}
                        day: 10
                        month: 7
                        year: 2020
                    unleaded: "107.70"
                c_george: true
                c_georgeEssentials: false
                c_groceryClickAndCollect: true
                c_groceryClickAndCollectImage: {image: {height: 1080,…}}
                    image: {height: 1080,…}
                        height: 1080
                        sourceUrl: "https://p20.zdusercontent.com/attachment/58910/gDBXr12Pgnpbuo5pjdnenBscP?token=eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..elcmPhwVI236Uq70DHK-0g.4wX3LoIZBvmeZ8XufZRN95wu_eNx_hQGuZ7Uq7jfwpkhi-7orwPfNKuv8tc5_KFS-UJ3tuiwVRT8HwqIet2PXeT6jXE4w5nHIF7KhoHVBGMu42yauAljxhppLBJb_oFPt89qxHaO5CQNFkaLZsfzV6-i2F26FmglJs0n_34LF4mHOQvASophHAA8SSmIa5rlGUeqrcFd3F1jGjGqq_8UWyOojF9xDgoXqfrZNaqlJCSwC89l7UkLsqtXNAWBLWDswsP7GNYySLzUk5_ut5aAg_DwHDOw2kDiUBJRo35OJuU.MrcEow8y-L3a-zUT-0rR4g"
                        thumbnails: [,…]
                            0: {height: 1068, url: "//dynl.mktgcdn.com/p/fyIydzdnhOol40zGaC_WdaSwnBCoZyi2MLLWGzafnVw/1900x1068.jpg",…}
                                height: 1068
                                url: "//dynl.mktgcdn.com/p/fyIydzdnhOol40zGaC_WdaSwnBCoZyi2MLLWGzafnVw/1900x1068.jpg"
                                width: 1900
                            1: {height: 348, url: "//dynl.mktgcdn.com/p/fyIydzdnhOol40zGaC_WdaSwnBCoZyi2MLLWGzafnVw/619x348.jpg",…}
                                height: 348
                                url: "//dynl.mktgcdn.com/p/fyIydzdnhOol40zGaC_WdaSwnBCoZyi2MLLWGzafnVw/619x348.jpg"
                                width: 619
                            2: {height: 337, url: "//dynl.mktgcdn.com/p/fyIydzdnhOol40zGaC_WdaSwnBCoZyi2MLLWGzafnVw/600x337.jpg",…}
                                height: 337
                                url: "//dynl.mktgcdn.com/p/fyIydzdnhOol40zGaC_WdaSwnBCoZyi2MLLWGzafnVw/600x337.jpg"
                                width: 600
                            3: {height: 110, url: "//dynl.mktgcdn.com/p/fyIydzdnhOol40zGaC_WdaSwnBCoZyi2MLLWGzafnVw/196x110.jpg",…}
                                height: 110
                                url: "//dynl.mktgcdn.com/p/fyIydzdnhOol40zGaC_WdaSwnBCoZyi2MLLWGzafnVw/196x110.jpg"
                                width: 196
                        url: "//dynl.mktgcdn.com/p/fyIydzdnhOol40zGaC_WdaSwnBCoZyi2MLLWGzafnVw/1920x1080.jpg"
                        width: 1920
                c_halal: false
                c_homeShopping: true
                c_hotChicken: false
                c_hoursSectionTitle: "Store Hours"
                c_instantPhotoPrint: true
                c_johnsonsDryCleaners: false
                c_linkedEntities: ["4383_C"]
                    0: "4383_C"
                c_maxSpeilman: false
                c_mcdonalds: false
                c_mcgeesButchers: false
                c_mobilePhoneConnectionCentre: false
                c_nearbyStoresSectionTitle: "Nearby Stores"
                c_opticians: false
                c_organic: false
                c_pagesURL: "https://storelocator.asda.com/east-of-england/norwich/hall-road"
                c_paypoint: false
                c_petrolStation: true
                c_pharmacy: false
                c_photoMe: false
                c_pizzaCounter: true
                c_postOffice: false
                c_promoSectionImage: {clickthroughUrl: "https://www.asdagoodliving.co.uk/food/get-your-cook-on",…}
                clickthroughUrl: "https://www.asdagoodliving.co.uk/food/get-your-cook-on"
                    image: {height: 666, thumbnails: [,…],…}
                    height: 666
                    thumbnails: [,…]
                        0: {height: 246, url: "//dynl.mktgcdn.com/p/Oca1qYoHmBBMTt250kCtsbUyx5Uf-eUuJYqVLHHUp0w/619x246.jpg",…}
                            height: 246
                            url: "//dynl.mktgcdn.com/p/Oca1qYoHmBBMTt250kCtsbUyx5Uf-eUuJYqVLHHUp0w/619x246.jpg"
                            width: 619
                        1: {height: 239, url: "//dynl.mktgcdn.com/p/Oca1qYoHmBBMTt250kCtsbUyx5Uf-eUuJYqVLHHUp0w/600x239.jpg",…}
                            height: 239
                            url: "//dynl.mktgcdn.com/p/Oca1qYoHmBBMTt250kCtsbUyx5Uf-eUuJYqVLHHUp0w/600x239.jpg"
                            width: 600
                        2: {height: 78, url: "//dynl.mktgcdn.com/p/Oca1qYoHmBBMTt250kCtsbUyx5Uf-eUuJYqVLHHUp0w/196x78.jpg",…}
                            height: 78
                            url: "//dynl.mktgcdn.com/p/Oca1qYoHmBBMTt250kCtsbUyx5Uf-eUuJYqVLHHUp0w/196x78.jpg"
                            width: 196
                    url: "//dynl.mktgcdn.com/p/Oca1qYoHmBBMTt250kCtsbUyx5Uf-eUuJYqVLHHUp0w/1670x666.jpg"
                    width: 1670
                c_recycling: false
                c_region: "Anglia"
                c_scanAndGo: true
                c_serviceDetailCardCoinstar: {hours: null, photoField(single): null, text(multi-line): null, text(single-line)1: "Coinstar",…}
                    hours: null
                    photoField(single): null
                    text(multi-line): null
                    text(single-line)1: "Coinstar"
                    text(single-line)2: null
                    url: null
                c_serviceDetailCardCosta: {hours: null, photoField(single): null, text(multi-line): null, text(single-line)1: "Costa Coffee",…}
                    hours: null
                    photoField(single): null
                    text(multi-line): null
                    text(single-line)1: "Costa Coffee"
                    text(single-line)2: null
                    url: null
                c_serviceDetailCardGeorge: {hours: null, photoField(single): null,…}
                    hours: null
                    photoField(single): null
                    text(multi-line): "Visit George to discover clothing, home accessories and footwear for the whole family, including school uniforms, women’s dresses and accessories."
                    text(single-line)1: "George"
                    text(single-line)2: "Shop now"
                    url: "http://www.george.com"
                c_serviceDetailCardGeorgeClickAndCollect: {hours: {holidayHours: [],…}, photoField(single): null,…}
                    hours: {holidayHours: [],…}
                        holidayHours: []
                        normalHours: [{day: "MONDAY", intervals: [{end: 2200, start: 800}], isClosed: false},…]
                            0: {day: "MONDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                                day: "MONDAY"
                                intervals: [{end: 2200, start: 800}]
                                    0: {end: 2200, start: 800}
                                        end: 2200
                                        start: 800
                                isClosed: false
                            1: {day: "TUESDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                                day: "TUESDAY"
                                intervals: [{end: 2200, start: 800}]
                                    0: {end: 2200, start: 800}
                                        end: 2200
                                        start: 800
                                isClosed: false
                            2: {day: "WEDNESDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                                day: "WEDNESDAY"
                                intervals: [{end: 2200, start: 800}]
                                    0: {end: 2200, start: 800}
                                        end: 2200
                                        start: 800
                                isClosed: false
                            3: {day: "THURSDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                                day: "THURSDAY"
                                intervals: [{end: 2200, start: 800}]
                                    0: {end: 2200, start: 800}
                                        end: 2200
                                        start: 800
                                isClosed: false
                            4: {day: "FRIDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                                day: "FRIDAY"
                                intervals: [{end: 2200, start: 800}]
                                    0: {end: 2200, start: 800}
                                        end: 2200
                                        start: 800
                                isClosed: false
                            5: {day: "SATURDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                                day: "SATURDAY"
                                intervals: [{end: 2200, start: 800}]
                                    0: {end: 2200, start: 800}
                                        end: 2200
                                        start: 800
                                isClosed: false
                            6: {day: "SUNDAY", intervals: [{end: 1600, start: 1000}], isClosed: false}
                                day: "SUNDAY"
                                intervals: [{end: 1600, start: 1000}]
                                    0: {end: 1600, start: 1000}
                                        end: 1600
                                        start: 1000
                                isClosed: false
                    photoField(single): null
                    text(multi-line): "TO ASSIST OUR STORE COLLEAGUES AT THIS BUSY TIME, THIS SERVICE IS NOT CURRENTLY AVAILABLE"
                    text(single-line)1: "George Click & Collect"
                    text(single-line)2: null
                    url: null
                c_serviceDetailCardPetrolFillingStation: {hours: null, photoField(single): null, text(multi-line): null,…}
                    hours: null
                    photoField(single): null
                    text(multi-line): null
                    text(single-line)1: "Petrol Filling Station"
                    text(single-line)2: null
                    url: null
                c_serviceDetailCardRugDoctor: {hours: null, photoField(single): null, text(multi-line): null, text(single-line)1: "Rug Doctor",…}
                    hours: null
                    photoField(single): null
                    text(multi-line): null
                    text(single-line)1: "Rug Doctor"
                    text(single-line)2: null
                    url: null
                c_serviceDetailCardScanAndGo: {hours: {holidayHours: [],…}, photoField(single): null,…}
                    hours: {holidayHours: [],…}
                        holidayHours: []
                        normalHours: [{day: "MONDAY", intervals: [{end: 2200, start: 800}], isClosed: false},…]
                            0: {day: "MONDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                            day: "MONDAY"
                            intervals: [{end: 2200, start: 800}]
                                0: {end: 2200, start: 800}
                                    end: 2200
                                    start: 800
                            isClosed: false
                        1: {day: "TUESDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                            day: "TUESDAY"
                            intervals: [{end: 2200, start: 800}]
                                0: {end: 2200, start: 800}
                                    end: 2200
                                    start: 800
                            isClosed: false
                        2: {day: "WEDNESDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                            day: "WEDNESDAY"
                            intervals: [{end: 2200, start: 800}]
                                0: {end: 2200, start: 800}
                                    end: 2200
                                    start: 800
                            isClosed: false
                        3: {day: "THURSDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                            day: "THURSDAY"
                            intervals: [{end: 2200, start: 800}]
                                0: {end: 2200, start: 800}
                                    end: 2200
                                    start: 800
                            isClosed: false
                        4: {day: "FRIDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                            day: "FRIDAY"
                            intervals: [{end: 2200, start: 800}]
                                0: {end: 2200, start: 800}
                                    end: 2200
                                    start: 800
                            isClosed: false
                        5: {day: "SATURDAY", intervals: [{end: 2200, start: 800}], isClosed: false}
                            day: "SATURDAY"
                            intervals: [{end: 2200, start: 800}]
                                0: {end: 2200, start: 800}
                                    end: 2200
                                    start: 800
                            isClosed: false
                        6: {day: "SUNDAY", intervals: [{end: 1600, start: 1000}], isClosed: false}
                            day: "SUNDAY"
                            intervals: [{end: 1600, start: 1000}]
                                0: {end: 1600, start: 1000}
                                    end: 1600
                                    start: 1000
                            isClosed: false
                    photoField(single): null
                    text(multi-line): "You can use both our Scan & Go handsets and Mobile App in this store."
                    text(single-line)1: "Scan & Go"
                    text(single-line)2: "Read More"
                    url: "https://www.asda.com/about/instore/scan-and-go"
                c_serviceDetailCardToYou: {hours: null, photoField(single): null,…}
                    hours: null
                    photoField(single): null
                    text(multi-line): "Click/Collect/Return at Asda stores with over a 100 of your favourite retailers."
                    text(single-line)1: "toyou"
                    text(single-line)2: "Read More"
                    url: "https://www.asda.com/toyou/?cmpid=dmc-_-ahc-_-vanityurl-_-generic-_-toyou&utm_source=vanityurl&utm_medium=dmc&utm_term=toyou&utm_content=generic&utm_campaign=ahc"
                c_serviceDetailCardsJustEatKitchen: {hours: null, photoField(single): null,…}
                    hours: null
                    photoField(single): null
                    text(multi-line): "Hot food for collection & delivery with Just Eat"
                    text(single-line)1: "Just Eat Kitchen"
                    text(single-line)2: null
                    url: null
                c_serviceDetailsTitle: "Services"
                c_storeLocatorFacilitiesFilter: ["ToYou", "Electric Car Charging", "Cash Machine"]
                    0: "ToYou"
                    1: "Electric Car Charging"
                    2: "Cash Machine"
                c_storeLocatorServicesFilter: ["Scan and Go", "Cafe", "Grocery Click & Collect", "Just Eat Kitchen"]
                    0: "Scan and Go"
                    1: "Cafe"
                    2: "Grocery Click & Collect"
                    3: "Just Eat Kitchen"
                c_storeType: ["Superstore"]
                    0: "Superstore"
                c_subway: false
                c_takeawayHotPizza: false
                c_technologyCentre: false
                c_thomasCook: false
                c_timpsons: false
                c_travelMoney: false
                c_travelMoneyClickAndCollect: false
                c_trolleySectionDescription: "Have you found an Asda trolley in your community that has been taken from our store? Let us know exactly where it is and we’ll come and collect it and return it back to Asda."
                c_trolleywise: true
                c_tuiTravel: false
                c_uKRegion: ["England"]
                    0: "England"
                c_whatsAtThisStoreList: ["Rotisserie", "Pizza Counter", "Food To Go", "Instant Photo Print", "Bakery", "Express Diner",…]
                    0: "Rotisserie"
                    1: "Pizza Counter"
                    2: "Food To Go"
                    3: "Instant Photo Print"
                    4: "Bakery"
                    5: "Express Diner"
                    6: "Helium Balloons"
                c_whatsAtThisStoreTitle: "What's at this store"
                cityCoordinate: {lat: 52.63529968261719, long: 1.294010043144226}
                    lat: 52.63529968261719
                    long: 1.294010043144226
                description: "Asda Norwich Hall Road Superstore is part of the Asda group; one of Britain’s leading grocery retailers. Asda offers great prices and quality products helping customers save money & live better. Visit us in-store and get groceries delivered to anywhere in the UK, or collect from your local store or petrol station at a convenient time for you."
                displayCoordinate: {lat: 52.606659, long: 1.288596}
                    lat: 52.606659
                    long: 1.288596
                facebookCoverPhoto: {image: {height: 273,…}}
                    image: {height: 273,…}
                        height: 273
                        sourceUrl: "https://scontent-bos3-1.xx.fbcdn.net/v/t1.0-9/s720x720/96379594_3804414072963302_8747344537808011264_n.png?_nc_cat=1&_nc_sid=dd9801&_nc_ohc=nKEKKHjU_hkAX-q129p&_nc_ht=scontent-bos3-1.xx&oh=ab10e811c6c006d6288d6351d4e576de&oe=5EE042E9"
                        url: "//dynl.mktgcdn.com/p/2aa3EsjnMcrzNB-76pyJ-NkPNOEPPc4tHnk2EYUXytA/720x273.png"
                        width: 720
                facebookName: "Asda"
                facebookPageUrl: "https://www.facebook.com/457762304577879"
                facebookParentPageId: "141700922567987"
                facebookProfilePhoto: {image: {height: 200,…}}
                    image: {height: 200,…}
                        height: 200
                        sourceUrl: "https://scontent-bos3-1.xx.fbcdn.net/v/t31.0-1/p200x200/17359452_1454805254590874_1187074472493250060_o.png?_nc_cat=1&_nc_sid=0c64ff&_nc_ohc=24w1xzbZDC4AX-30l-y&_nc_ht=scontent-bos3-1.xx&oh=4cff6b231ae9659e4873db25995cf638&oe=5EE05BEA"
                        url: "//dynl.mktgcdn.com/p/RB41DA38HIa7bG3QCUXHyDXHx0mdAt8i80htQFuQqos/200x200.png"
                        width: 200
                featuredMessage: {description: "Visit us in-store or online", url: "https://www.asda.com/"}
                    description: "Visit us in-store or online"
                    url: "https://www.asda.com/"
                geocodedCoordinate: {lat: 52.6065825, long: 1.2876707}
                    lat: 52.6065825
                    long: 1.2876707
                geomodifier: "Norwich Hall Road"
                googleAccountId: "117489345900231741701"
                googleAttributes: {}
                googleCoverPhoto: {,…}
                    image: {height: 720, sourceUrl: "https://yext.box.com/shared/static/0oq6t16l8qtmkvbedwbwqt90lm3cv98q.jpg",…}
                        height: 720
                        sourceUrl: "https://yext.box.com/shared/static/0oq6t16l8qtmkvbedwbwqt90lm3cv98q.jpg"
                        thumbnails: [,…]
                            0: {height: 348, url: "//dynl.mktgcdn.com/p/Oyr6H_jd6d6J9STzXi32HJoiX89URA7GxW1eOl31th0/619x348.jpg",…}
                                height: 348
                                url: "//dynl.mktgcdn.com/p/Oyr6H_jd6d6J9STzXi32HJoiX89URA7GxW1eOl31th0/619x348.jpg"
                                width: 619
                            1: {height: 337, url: "//dynl.mktgcdn.com/p/Oyr6H_jd6d6J9STzXi32HJoiX89URA7GxW1eOl31th0/600x337.jpg",…}
                                height: 337
                                url: "//dynl.mktgcdn.com/p/Oyr6H_jd6d6J9STzXi32HJoiX89URA7GxW1eOl31th0/600x337.jpg"
                                width: 600
                            2: {height: 110, url: "//dynl.mktgcdn.com/p/Oyr6H_jd6d6J9STzXi32HJoiX89URA7GxW1eOl31th0/196x110.jpg",…}
                                height: 110
                                url: "//dynl.mktgcdn.com/p/Oyr6H_jd6d6J9STzXi32HJoiX89URA7GxW1eOl31th0/196x110.jpg"
                                width: 196
                            url: "//dynl.mktgcdn.com/p/Oyr6H_jd6d6J9STzXi32HJoiX89URA7GxW1eOl31th0/1280x720.jpg"
                            width: 1280
                googlePlaceId: "ChIJKZs0vAHk2UcRkaLqm7RF3hk"
                hours: {holidayHours: [],…}
                    holidayHours: []
                    normalHours: [{day: "MONDAY", intervals: [{end: 2200, start: 700}], isClosed: false},…]
                        0: {day: "MONDAY", intervals: [{end: 2200, start: 700}], isClosed: false}
                            day: "MONDAY"
                            intervals: [{end: 2200, start: 700}]
                                0: {end: 2200, start: 700}
                                    end: 2200
                                    start: 700
                            isClosed: false
                        1: {day: "TUESDAY", intervals: [{end: 2200, start: 700}], isClosed: false}
                            day: "TUESDAY"
                            intervals: [{end: 2200, start: 700}]
                                0: {end: 2200, start: 700}
                                    end: 2200
                                    start: 700
                            isClosed: false
                        2: {day: "WEDNESDAY", intervals: [{end: 2200, start: 700}], isClosed: false}
                            day: "WEDNESDAY"
                            intervals: [{end: 2200, start: 700}]
                                0: {end: 2200, start: 700}
                                    end: 2200
                                    start: 700
                            isClosed: false
                        3: {day: "THURSDAY", intervals: [{end: 2200, start: 700}], isClosed: false}
                            day: "THURSDAY"
                            intervals: [{end: 2200, start: 700}]
                                0: {end: 2200, start: 700}
                                    end: 2200
                                    start: 700
                            isClosed: false
                        4: {day: "FRIDAY", intervals: [{end: 2200, start: 700}], isClosed: false}
                            day: "FRIDAY"
                            intervals: [{end: 2200, start: 700}]
                                0: {end: 2200, start: 700}
                                    end: 2200
                                    start: 700
                            isClosed: false
                        5: {day: "SATURDAY", intervals: [{end: 2200, start: 700}], isClosed: false}
                            day: "SATURDAY"
                            intervals: [{end: 2200, start: 700}]
                                0: {end: 2200, start: 700}
                                    end: 2200
                                    start: 700
                            isClosed: false
                        6: {day: "SUNDAY", intervals: [{end: 1600, start: 1000}], isClosed: false}
                            day: "SUNDAY"
                            intervals: [{end: 1600, start: 1000}]
                                0: {end: 1600, start: 1000}
                                    end: 1600
                                    start: 1000
                            isClosed: false
                isoRegionCode: "NFK"
                logo: {image: {height: 400, thumbnails: [,…],…}}
                    image: {height: 400, thumbnails: [,…],…}
                        height: 400
                        thumbnails: [,…]
                        0: {height: 150, url: "//dynl.mktgcdn.com/p/uxpSIwyZRALdFsUMpGERiKVVeUVlEaMMTBvKbuOZB-E/150x150.png",…}
                            height: 150
                            url: "//dynl.mktgcdn.com/p/uxpSIwyZRALdFsUMpGERiKVVeUVlEaMMTBvKbuOZB-E/150x150.png"
                            width: 150
                        1: {height: 94, url: "//dynl.mktgcdn.com/p/uxpSIwyZRALdFsUMpGERiKVVeUVlEaMMTBvKbuOZB-E/94x94.png",…}
                            height: 94
                            url: "//dynl.mktgcdn.com/p/uxpSIwyZRALdFsUMpGERiKVVeUVlEaMMTBvKbuOZB-E/94x94.png"
                            width: 94
                        url: "//dynl.mktgcdn.com/p/uxpSIwyZRALdFsUMpGERiKVVeUVlEaMMTBvKbuOZB-E/400x400.png"
                        width: 400
                mainPhone: {countryCode: "GB", display: "01603 964246", number: "+441603964246"}
                    countryCode: "GB"
                    display: "01603 964246"
                    number: "+441603964246"
                matchingSearchIds: ["4656"]
                    0: "4656"
                meta: {accountId: "3332574490752748386", countryCode: "GB", entityType: "location", folderId: "298473",…}
                    accountId: "3332574490752748386"
                    countryCode: "GB"
                    entityType: "location"
                    folderId: "298473"
                    id: "4383"
                    labels: ["78691", "78687", "78686", "78690", "82562", "84251", "95837"]
                        0: "78691"
                        1: "78687"
                        2: "78686"
                        3: "78690"
                        4: "82562"
                        5: "84251"
                        6: "95837"
                    language: "en_GB"
                    schemaTypes: ["GroceryStore", "AutomatedTeller", "GasStation", "ClothingStore"]
                        0: "GroceryStore"
                        1: "AutomatedTeller"
                        2: "GasStation"
                        3: "ClothingStore"
                    timestamp: "2020-07-13T01:47:47"
                    uid: "6wOk6E"
                    utcOffsets: [{offset: 3600, start: 1585443600}, {offset: 0, start: 1603587600}, {offset: 3600, start: 1616893200}]
                        0: {offset: 3600, start: 1585443600}
                            offset: 3600
                            start: 1585443600
                        1: {offset: 0, start: 1603587600}
                            offset: 0
                            start: 1603587600
                        2: {offset: 3600, start: 1616893200}
                            offset: 3600
                            start: 1616893200
                    yextId: 13618640
                name: "Asda Norwich Hall Road Superstore"
                timezone: "Europe/London"
                websiteUrl: "http://storelocator.asda.com/store/norwich-hall-road"
                yextDisplayCoordinate: {lat: 52.6065825, long: 1.2876707}
                lat: 52.6065825
                long: 1.2876707
                yextRoutableCoordinate: {lat: 52.60555, long: 1.28634}
                lat: 52.60555
                long: 1.28634
            url: "east-of-england/norwich/hall-road"
        1: {derivedData: {address: {countryName: "United Kingdom"}, uber: {,…}},…}
    geo: {address: {city: "Norwich", country: "GB", postalCode: "NR3 1TP", region: "England"},…}
schema: {additionalHoursText: {archived: false, displayName: "Additional Hours Text",…},…}
searchId: "entitysearch"
siteAttributes: {footer: {legal: "© 2019 ASDA. All Rights Reserved.", link1: "Privacy", link2: "Terms of Service",…},…}
siteDomain: "storelocator.asda.com"
siteId: 1812
siteInternalHostName: "storelocator.asda.com"
versions: {en: "index.html"}

## Code for retreiving ASDA's opening hours data:

In [ ]:
def get_asda_data(lat,lng):
    import json
    API_URL = "https://storelocator.asda.com/index.html"
    params = { 'q' : "{0},{1}".format(lat,lng)}
    headers = {
    "accept": "application/json",
  }
    rq = requests.get(API_URL,params=params, headers = headers)
    if rq.status_code != 200:
        print(rq.json())
        return False
    # res = json.dumps(rq.text)
    try:
        res = rq.json()["response"]
        if res["count"] == 0:
            return False
    except:
        return False
    i = 0 
    #INSERT POTENTIAL CHECK THAT STORE MATCHES DESIRED STORENAME!! (res["entities"][0]["name"])
    #e.g.:
    # while True:
    #     if name != res["entities"][i]["name"]:
    #         i += 1
    #     else:
    #         break
    openingHours = res["entities"][i]["profile"]["hours"]["normalHours"]
    hoursArray = []
    dayKeys = { 0 : 'Monday', 1 : 'Tuesday', 2 : 'Wednesday', 3 : 'Thursday', 4 : 'Friday', 5 : 'Saturday', 6 : 'Sunday'}
    for index in range(len(openingHours)):
        if openingHours[index]["isClosed"] != False:
            closedDayHours = {  'day' : dayKeys[index],
                                'open' : False  }
            hoursArray.append(closedDayHours)
        actualHours = []
        for timeSlotNumber in range(len(openingHours[index]['intervals'])):
            try:
                actualHoursDict = {}
                start = str(openingHours[index]['intervals'][timeSlotNumber]['start'])
                if start == "0":
                    start = "00:00"
                elif len(start) == 3:
                    start = "0{0}:{1}".format(start[0],start[1:])
                else:
                    start = "{0}:{1}".format(start[:2],start[2:])
                end = str(openingHours[index]['intervals'][timeSlotNumber]['end'])
                if end == "0":
                    end = "00:00"
                elif len(end) == 3:
                    end = "0{0}:{1}".format(end[0],end[1:])
                else:
                    end = "{0}:{1}".format(end[:2],end[2:])
                actualHoursDict['open'] = start
                actualHoursDict['close'] = end
                actualHours.append(actualHoursDict)
            except:
                #If dictionary keys start_time or end_time dont exist, assume store is closed on that day
                closedDayHours = {  'day' : dayKeys[index],
                                'open' : False  }
                hoursArray.append(closedDayHours)
                break
        keyHours = {'day' : dayKeys[index],
                'open' : True,
                'hours' : actualHours
                }
        hoursArray.append(keyHours)
    return hoursArray

# Tesco's API

## Parameters:
* offset : 0 
* limit : number of stores to show (int)
* sort : <code>"near:%22{LATITUDE},{LONGITUDE}%22"</code>
* filter : <code>"category:Store%20AND%20isoCountryCode:x-uk"</code>
* fields : what to return (subset of <code>"name,geo,openingHours,altIds.branchNumber,contact,facilities"</code>)

## Sample headers:
<code>"headers": {
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "traceid": "c78bc741-5b7b-4797-9aa3-bf3a2f96f7a4",
    "x-appkey": "store-locator-web-cde"
  }</code>
  
 ### HOWEVER:
 One appears to be able to bypass any forms of authentication (?) by simply using the following headers:
 
 <code>"headers": {"x-appkey": "store-locator-web-cde"}</code>
 
## Sample API Call directly from website:
<code>rq = requests.get("https://api.tesco.com/tescolocation/v3/locations/search?offset=0&limit=10&sort=near:%2252.62682342529297,1.3366719484329224%22&filter=category:Store%20AND%20isoCountryCode:x-uk&fields=name,geo,openingHours,altIds.branchNumber,contact,facilities", headers={"x-appkey": "store-locator-web-cde"})
</code>

RESULTS:
* of interest is: rq.json()['results'][0]['location']['openingHours'][0]['standardOpeningHours']
* and : rq.json()['results'][0]['location']['openingHours'][0]['exceptions']
* However, exceptions provides you a historical(?) list of dictionaries of all dates that have had exceptions in the past(?), in a seemingly completely unsorted order, making using this information for anything other than extrapolation (**NOT IMPLEMENTED**) of 'special dates'' opening times (e.g. christmas) somewhat unfeasible.

### Sample response:

In [ ]:
{,…}
    results: [{location: {id: "2ad4c578-962b-42cd-8541-0f75c1952680", name: "Norwich Plumstead Rd Express",…},…},…]
        0: {location: {id: "2ad4c578-962b-42cd-8541-0f75c1952680", name: "Norwich Plumstead Rd Express",…},…}
            distanceFrom: {unit: "miles", value: 0.8}
                unit: "miles"
                value: 0.8
            location: {id: "2ad4c578-962b-42cd-8541-0f75c1952680", name: "Norwich Plumstead Rd Express",…}
                altIds: {branchNumber: 5284}
                    branchNumber: 5284
                contact: {address: {lines: [{lineNumber: 1, text: "197 Plumstead Rd"}], town: "Norwich", postcode: "NR1 4AB"},…}
                    address: {lines: [{lineNumber: 1, text: "197 Plumstead Rd"}], town: "Norwich", postcode: "NR1 4AB"}
                        lines: [{lineNumber: 1, text: "197 Plumstead Rd"}]
                            0: {lineNumber: 1, text: "197 Plumstead Rd"}
                                lineNumber: 1
                                text: "197 Plumstead Rd"
                            postcode: "NR1 4AB"
                            town: "Norwich"
                    phoneNumbers: [{alias: "Default", number: "0345 674 6492"}]
                        0: {alias: "Default", number: "0345 674 6492"}
                            alias: "Default"
                            number: "0345 674 6492"
                facilities: [{name: "FREE_FROM", tags: ["public", "food_range"], description: "Free From"},…]
                0: {name: "FREE_FROM", tags: ["public", "food_range"], description: "Free From"}
                    description: "Free From"
                    name: "FREE_FROM"
                    tags: ["public", "food_range"]
                        0: "public"
                        1: "food_range"
                1: {name: "ATM", tags: ["public"], description: "ATM"}
                    description: "ATM"
                    name: "ATM"
                    tags: ["public"]
                        0: "public"
                2: {name: "GIFT_CARDS", tags: ["public"], description: "Sells Gift Cards"}
                    description: "Sells Gift Cards"
                    name: "GIFT_CARDS"
                    tags: ["public"]
                        0: "public"
                3: {name: "TESCO_PAY_PLUS", tags: ["public"], description: "Tesco Pay+"}
                    description: "Tesco Pay+"
                    name: "TESCO_PAY_PLUS"
                    tags: ["public"]
                        0: "public"
                4: {name: "ASSISTANCE_DOGS", tags: ["enabling", "public"],…}
                    description: "Assistance dogs are welcome in our store."
                    name: "ASSISTANCE_DOGS"
                    tags: ["enabling", "public"]
                        0: "enabling"
                        1: "public"
                5: {name: "PAYPOINT", tags: ["public"], description: "PayPoint"}
                    description: "PayPoint"
                    name: "PAYPOINT"
                    tags: ["public"]
                        0: "public"
                geo: {coordinates: {longitude: 1.32715, latitude: 52.63697}}
                    coordinates: {longitude: 1.32715, latitude: 52.63697}
                        latitude: 52.63697
                        longitude: 1.32715
                id: "2ad4c578-962b-42cd-8541-0f75c1952680"
                name: "Norwich Plumstead Rd Express"
                openingHours: [{type: "Trading",…}]
                    0: {type: "Trading",…}
                        exceptions: [{date: "2017-12-25", hours: {isOpen: "false"}},…]
                            0: {date: "2017-12-25", hours: {isOpen: "false"}}
                                date: "2017-12-25"
                                hours: {isOpen: "false"}
                                    isOpen: "false"
                            1: {date: "2020-03-21", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2020-03-21"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            2: {date: "2017-12-26", hours: {isOpen: "true", open: "0800", close: "2200"}}
                                date: "2017-12-26"
                                hours: {isOpen: "true", open: "0800", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0800"
                            3: {date: "2020-03-22", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2020-03-22"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            4: {date: "2020-03-18", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2020-03-18"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            5: {date: "2019-12-31", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2019-12-31"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            6: {date: "2018-12-24", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2018-12-24"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            7: {date: "2020-03-23", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2020-03-23"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            8: {date: "2019-12-25", hours: {isOpen: "false"}}
                                date: "2019-12-25"
                                hours: {isOpen: "false"}
                                    isOpen: "false"
                            9: {date: "2020-03-19", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2020-03-19"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            10: {date: "2018-01-01", hours: {isOpen: "true", open: "0800", close: "2200"}}
                                date: "2018-01-01"
                                hours: {isOpen: "true", open: "0800", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0800"
                            11: {date: "2018-12-31", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2018-12-31"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            12: {date: "2019-12-26", hours: {isOpen: "true", open: "0800", close: "2200"}}
                                date: "2019-12-26"
                                hours: {isOpen: "true", open: "0800", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0800"
                            13: {date: "2020-01-01", hours: {isOpen: "true", open: "0800", close: "2200"}}
                                date: "2020-01-01"
                                hours: {isOpen: "true", open: "0800", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0800"
                            14: {date: "2018-12-25", hours: {isOpen: "false"}}
                                date: "2018-12-25"
                                hours: {isOpen: "false"}
                                    isOpen: "false"
                            15: {date: "2017-12-24", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2017-12-24"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            16: {date: "2019-12-24", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2019-12-24"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            17: {date: "2017-12-31", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2017-12-31"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            18: {date: "2019-01-01", hours: {isOpen: "true", open: "0800", close: "2200"}}
                                date: "2019-01-01"
                                hours: {isOpen: "true", open: "0800", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0800"
                            19: {date: "2020-03-20", hours: {isOpen: "true", open: "0600", close: "2200"}}
                                date: "2020-03-20"
                                hours: {isOpen: "true", open: "0600", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0600"
                            20: {date: "2018-12-26", hours: {isOpen: "true", open: "0800", close: "2200"}}
                                date: "2018-12-26"
                                hours: {isOpen: "true", open: "0800", close: "2200"}
                                    close: "2200"
                                    isOpen: "true"
                                    open: "0800"
                        standardOpeningHours: {mo: {isOpen: "true", open: "0600", close: "2300"}, tu: {isOpen: "true", open: "0600", close: "2300"},…}
                            fr: {isOpen: "true", open: "0600", close: "2300"}
                                close: "2300"
                                isOpen: "true"
                                open: "0600"
                            mo: {isOpen: "true", open: "0600", close: "2300"}
                                close: "2300"
                                isOpen: "true"
                                open: "0600"
                            sa: {isOpen: "true", open: "0600", close: "2300"}
                                close: "2300"
                                isOpen: "true"
                                open: "0600"
                            su: {isOpen: "true", open: "0600", close: "2300"}
                                close: "2300"
                                isOpen: "true"
                                open: "0600"
                            th: {isOpen: "true", open: "0600", close: "2300"}
                                close: "2300"
                                isOpen: "true"
                                open: "0600"
                            tu: {isOpen: "true", open: "0600", close: "2300"}
                                close: "2300"
                                isOpen: "true"
                                open: "0600"
                            we: {isOpen: "true", open: "0600", close: "2300"}
                                close: "2300"
                                isOpen: "true"
                                open: "0600"
                        type: "Trading"
        1: {location: {id: "122a682a-34df-45f3-92d7-0cd3c4326067", name: "Norwich Wales Rd Express",…},…}
        2: {location: {id: "8c7b5bce-0139-4831-8f58-7280d88f6ed4", name: "Norwich Westlegate Express",…},…}
    
    

 # Code for retrieving Tesco's opening times data:

In [ ]:
def get_tesco_data(lat, lng):
    API_URL = "https://api.tesco.com/tescolocation/v3/locations/search?offset=0"
    params =    { 'limit' : limit,
                'sort' : "near:%22{0},{1}%22".format(lat,lng),
                'filter' : "category:Store%20AND%20isoCountryCode:x-uk",
                'fields' : "name,geo,openingHours"
                #known fields: "name,geo,openingHours,altIds.branchNumber,contact,facilities"
                }
    headers = {"x-appkey": "store-locator-web-cde"}
    # rq = requests.get(API_URL, params=params, headers=headers)
    #(FOR SOME REASON THIS ^ LINE CAUSES ISSUES SO HAVE REPLACED WITH THE BELOW MANUAL LINE RATHER THAN USING THE PARAMS ARGUMENT OF requests.get())
    rq = requests.get(API_URL + "&limit={0}&sort={1}&filter={2}&fields={3}".format(params['limit'], params['sort'], params['filter'], params['fields']), headers=headers)
    if rq.status_code != 200:
        print(rq.json())
        return False
    res = rq.json()['results']
    i=0
    #INSERT POTENTIAL CHECK THAT STORE MATCHES DESIRED STORENAME!! (res[0]['location']['name'])
    #e.g.:
    # while True:
    #     if name != res[i]['location']['name']:
    #         i += 1
    #     else:
    #         break
    openingHours = res[i]['location']['openingHours'][0]['standardOpeningHours']
    dayKeys = { 'mo' : 'Monday', 'tu' : 'Tuesday', 'we' : 'Wednesday', 'th' : 'Thursday', 'fr' : 'Friday', 'sa' : 'Saturday', 'su' : 'Sunday'}
    hoursArray = []
    for key in openingHours:

        if openingHours[key]['isOpen'] == 'true':
            actualHours = {'open' : openingHours[key]['open'][:2] + ":" + openingHours[key]['open'][2:],
                        'close' : openingHours[key]['close'][:2] + ":" + openingHours[key]['close'][2:]}
            keyHours = {'day' : dayKeys[key],
                    'open' : True,
                    'hours' : [actualHours]
                    }
        else:
            keyHours = {'day' : dayKeys[key],
                    'open' : False}

        hoursArray.append(keyHours)
    return hoursArray

# Morrison's API
## Base URL:
https://api.morrisons.com/location/v2//stores
## Parameters:
* distance = not sure of units, website uses 50000 and returns at least 21 miles radius, so one assumes meters, so 50km radius.
* lat = latitude
* lon = longitude
* limit = number of stores to display
* offset = (?) wesbite uses 0
* storeformat = "supermarket"

## Headers:
* apikey = one i used on first day: "kxBdM2chFwZjNvG2PwnSn3sj6C53dLEY"

## Sample API Call:
<code>https://api.morrisons.com/location/v2//stores?apikey=kxBdM2chFwZjNvG2PwnSn3sj6C53dLEY&distance=50000&lat=52.62682342529297&limit=10&lon=1.3366719484329224&offset=0&storeformat=supermarket</code>
### Sample Response:

In [1]:
{,…}
    metadata: {limit: 10, count: 10, offset: 0, total: 12}
    stores: [{name: 115, storeName: "Norwich", storeFormat: "supermarket", category: "Supermarket",…},…]
        0: {name: 115, storeName: "Norwich", storeFormat: "supermarket", category: "Supermarket",…}
            address: {addressLine1: "4 Albion Way", addressLine2: "Riverside", county: "Norfolk", city: "Norwich",…}
                addressLine1: "4 Albion Way"
                addressLine2: "Riverside"
                city: "Norwich"
                country: "United Kingdom"
                county: "Norfolk"
                postcode: "NR1 1WR"
            category: "Supermarket"
            distance: 2065
            location: {latitude: 52.623662, longitude: 1.30663}
                latitude: 52.623662
                longitude: 1.30663
            name: 115
            openingTimes: {mon: {open: "07:00:00", close: "22:00:00"}, tue: {open: "07:00:00", close: "22:00:00"},…}
                fri: {open: "07:00:00", close: "22:00:00"}
                    close: "22:00:00"
                    open: "07:00:00"
                mon: {open: "07:00:00", close: "22:00:00"}
                    close: "22:00:00"
                    open: "07:00:00"
                sat: {open: "07:00:00", close: "22:00:00"}
                    close: "22:00:00"
                    open: "07:00:00"
                sun: {open: "10:00:00", close: "16:00:00"}
                    close: "16:00:00"
                    open: "10:00:00"
                thu: {open: "07:00:00", close: "22:00:00"}
                    close: "22:00:00"
                    open: "07:00:00"
                tue: {open: "07:00:00", close: "22:00:00"}
                    close: "22:00:00"
                    open: "07:00:00"
                wed: {open: "07:00:00", close: "22:00:00"}
                    close: "22:00:00"
                    open: "07:00:00"
            region: "East England"
            satnav: {latitude: 52.623662, longitude: 1.30663}
                latitude: 52.623662
                longitude: 1.30663
            storeFormat: "supermarket"
            storeName: "Norwich"
            telephone: "01603 663999"
        1: {name: 385, storeName: "Catton", storeFormat: "supermarket", category: "Supermarket",…}
        2: {name: 655, storeName: "Wymondham", storeFormat: "supermarket", category: "Supermarket",…}
        3: {name: 275, storeName: "Beccles", storeFormat: "supermarket", category: "Supermarket",…}


SyntaxError: invalid syntax (<ipython-input-1-3db5c2fcad70>, line 1)

# Code for retrieving Morrison's opening hours data:

In [ ]:
def get_morrisons_data(lat, lng):
    apikey = "kxBdM2chFwZjNvG2PwnSn3sj6C53dLEY"
    API_URL = "https://api.morrisons.com/location/v2//stores"
    params = { 'apikey' : apikey,
                'distance' : radius*1000,
                'lat' : lat,
                'lon' : lng,
                'limit' : limit,
                'offset' : 0,
                'storeformat' : "supermarket"
             }
    rq = requests.get(API_URL, params=params)
    if rq.status_code != 200:
        print(rq.json())
        return False
    try:
        res = rq.json()["stores"]
        if len(res) == 0:
            return False
    except:
        return False
    i=0
    #INSERT POTENTIAL CHECK THAT STORE MATCHES DESIRED STORENAME!! (e.g. "Morrisons" + res[0]["storename"])
    #e.g.:
    # while True:
    #     if name != "Morrisons" + res[i]["storename"]:
    #         i += 1
    #     else:
    #         break
    openingHours = res[i]['openingTimes']
    dayKeys = { 'mon' : 'Monday', 'tue' : 'Tuesday', 'wed' : 'Wednesday', 'thu' : 'Thursday', 'fri' : 'Friday', 'sat' : 'Saturday', 'sun' : 'Sunday'}
    hoursArray = []
    for key in ['mon','tue','wed','thu','fri','sat','sun']:
        try:
            actualHours = {'open' : openingHours[key]['open'][:5],
                        'close' : openingHours[key]['close'][:5]}
            keyHours = {'day' : dayKeys[key],
                    'open' : True,
                    'hours' : [actualHours]
                    }
        except:
            keyHours = {'day' : dayKeys[key],
                        'open' : False}

        hoursArray.append(keyHours)
    return hoursArray

# Waitrose's "API":
Waitrose's system works very differently from the majority of other supermarkets'.
Waitrose first calls their NearestBranchesCmd API, to obtain a list of the 6 nearest branches.
Waitrose then calls their StandardWorkingHoursView API for the closest 3 of those stores, which returns an html encoded table(?) of the opening hours of that branch (see examples below).
## NearestBranchesCmd API:
### Base URL:
https://www.waitrose.com/shop/NearestBranchesCmd
### Parameters:
* latitude = latitude
* longitude = longitude
* fromMultipleBranch = True (Note: True/False doesnt seem to make a difference) (optional)
* _ = 1594910860791 (on the first day i used it, perhaps this is some form of API key??) (optional)
### Sample API Call:
https://www.waitrose.com/shop/NearestBranchesCmd?latitude=52.6054482&longitude=1.2412015&fromMultipleBranch=true&_=1594910860791
https://www.waitrose.com/shop/NearestBranchesCmd?latitude=52.6470831&longitude=1.3433364&fromMultipleBranch=true&_=1594911360288
### Sample Response:

In [ ]:
{success: true, branchList: [{branchId: 222, branchName: "Norwich", phoneNumber: "01603 458114",…},…]}
    branchList: [{branchId: 222, branchName: "Norwich", phoneNumber: "01603 458114",…},…]
        0: {branchId: 222, branchName: "Norwich", phoneNumber: "01603 458114",…}
            addressLine1: "Eaton Centre, Church Lane"
            branchDesc: "Standard Branch"
            branchId: 222
            branchName: "Norwich"
            city: "Eaton"
            isEntertaining: false
            isGrocery: false
            latitude: 52.60794
            longitude: 1.25119
            phoneNumber: "01603 458114"
            postCode: "NR4 6NU"
        1: {branchId: 662, branchName: "Wymondham", phoneNumber: "01953 607772", addressLine1: "Norwich Road",…}
        2: {branchId: 622, branchName: "North Walsham", phoneNumber: "01692 503109", addressLine1: "Cromer Road",…}
        3: {branchId: 686, branchName: "Swaffham", phoneNumber: "01760 336307", addressLine1: "Castle Acre Road",…}
        4: {branchId: 483, branchName: "Saxmundham", phoneNumber: "01728 603004", addressLine1: "Church Street",…}
        5: {branchId: 140, branchName: "Bury St Edmunds", phoneNumber: "01284 725222",…}
    success: true

## StandardWorkingHours API:
### Base URL:
https://www.waitrose.com/shop/StandardWorkingHoursView
### Parameters:
* branchId = id of branch (retrieved from the NearestBranchesCmd API)
* storeId = id of store (retrieved from the NearestBranchesCmd API)
* linkType = "standard" (not sure what other options could be)
* _ = 1594911360644 (on first try, once again unsure of what this is or does, but does seem to go up over quite quickly time, so perhaps a global counter?)

### Sample API Call:
https://www.waitrose.com/shop/StandardWorkingHoursView?branchId=222&storeId=10317&linkType=standard&_=1594911360644
### Sample Response:
#### Note:
Edit this cell in order to see exact response received (the < pre > and < /pre > tags have been added in for formatting and were not part of the original reponse).
<pre>

<table>
<tbody>
	
			 <tr>
			 
			 <td class="day">
			 Sunday:
			 </td>
 		      
			  <td class="time">
			    10:00 - 16:00
			  </td>
			  
		</tr>
	
			 <tr>
			 
			 <td class="day">
			 Monday:
			 </td>
 		      
			  <td class="time">
			    07:00 - 20:00
			  </td>
			  
		</tr>
	
			 <tr>
			 
			 <td class="day">
			 Tuesday:
			 </td>
 		      
			  <td class="time">
			    07:00 - 20:00
			  </td>
			  
		</tr>
	
			 <tr>
			 
			 <td class="day">
			 Wednesday:
			 </td>
 		      
			  <td class="time">
			    07:00 - 20:00
			  </td>
			  
		</tr>
	
			 <tr>
			 
			 <td class="day">
			 Thursday:
			 </td>
 		      
			  <td class="time">
			    07:00 - 20:00
			  </td>
			  
		</tr>
	
			 <tr>
			 
			 <td class="day">
			 Friday:
			 </td>
 		      
			  <td class="time">
			    07:00 - 20:00
			  </td>
			  
		</tr>
	
			 <tr>
			 
			 <td class="day">
			 Saturday:
			 </td>
 		      
			  <td class="time">
			    07:00 - 20:00
			  </td>
			  
		</tr>
	
</tbody>
</table>

</pre>

### Version obtained in plaintext via opening in Chrome (although in browser this does display with newlines):
Sunday:	10:00 - 16:00
Monday:	07:00 - 20:00
Tuesday:	07:00 - 20:00
Wednesday:	07:00 - 20:00
Thursday:	07:00 - 20:00
Friday:	07:00 - 20:00
Saturday:	07:00 - 20:00

# Code for retreiving Waitrose's opening hours data:

In [ ]:
def get_waitrose_data(lat, lng):
    API_URL_locations = "https://www.waitrose.com/shop/NearestBranchesCmd"
    API_URL_hours = "https://www.waitrose.com/shop/StandardWorkingHoursView"
    params_locations = {'latitude' : lat,
                        'longitude' : lng,
                        }
    rq = requests.get(API_URL_locations, params=params_locations)
    if rq.status_code != 200:
        print(rq.json())
        return False
    try:
        res = rq.json()
        if not res['success'] or len(res['branchList']) == 0:
            return False
        res = res["branchList"]
    except:
        return False
    i=0
    #INSERT POTENTIAL CHECK THAT STORE MATCHES DESIRED STORENAME!! (e.g. "Waitrose" + res[0]["branchName"])
    #e.g.:
    # while True:
    #     if name != "Waitrose" + res[i]["branchName"]:
    #         i += 1
    #     else:
    #         break
    branchId = res[i]['branchId']
    params_hours = { 'branchId' : branchId}
    rq = requests.get(API_URL_hours, params=params_hours)
    hoursStringHTML = rq.text
    cleanr = re.compile('<.*?>|\s')
    hoursString = re.sub(cleanr, '', hoursStringHTML)
    #This section is to ensure if the shop is closed, correct times are displayed
    sundayIndex = hoursString.find("Sunday:")
    mondayIndex = hoursString.find("Monday:")
    tuesdayIndex = hoursString.find("Tuesday:")
    wednesdayIndex = hoursString.find("Wednesday:")
    thursdayIndex = hoursString.find("Thursday:")
    fridayIndex = hoursString.find("Friday:")
    saturdayIndex = hoursString.find("Saturday:")

    hoursArray = []
    #equivalent of the usual for loop V
    if mondayIndex == -1:
        keyHours = {'day' : "Monday",
                        'open' : False}
        hoursArray.append(keyHours)
    else:
        try:
            day = hoursString[mondayIndex+7:tuesdayIndex]
            if day.find("los") != -1: #check if the world close or Close or closed or Closed or Closure etc.. appears
                keyHours = {'day' : "Monday",
                        'open' : False}
                hoursArray.append(keyHours)
            firstDigitIndex = [x.isdigit() for x in day].index(True)
            actualHours = { 'open' : day[firstDigitIndex:firstDigitIndex+5],
                            'close' : day[firstDigitIndex+6:firstDigitIndex+11]}
            keyHours = {'day' : "Monday",
                        'open' : True,
                        'hours' : actualHours}
            hoursArray.append(keyHours)
        except:
            keyHours = {'day' : "Monday",
                        'open' : False}
            hoursArray.append(keyHours)
            

    if tuesdayIndex == -1:
        keyHours = {'day' : "Tuesday",
                        'open' : False}
        hoursArray.append(keyHours)
    else:
        try:
            day = hoursString[tuesdayIndex+8:wednesdayIndex]
            if day.find("los") != -1: #check if the world close or Close or closed or Closed or Closure etc.. appears
                keyHours = {'day' : "Tuesday",
                        'open' : False}
                hoursArray.append(keyHours)
            firstDigitIndex = [x.isdigit() for x in day].index(True)
            actualHours = { 'open' : day[firstDigitIndex:firstDigitIndex+5],
                            'close' : day[firstDigitIndex+6:firstDigitIndex+11]}
            keyHours = {'day' : "Tuesday",
                        'open' : True,
                        'hours' : actualHours}
            hoursArray.append(keyHours)
        except:
            keyHours = {'day' : "Tuesday",
                        'open' : False}
            hoursArray.append(keyHours)
    if wednesdayIndex == -1:
        keyHours = {'day' : "Wednesday",
                        'open' : False}
        hoursArray.append(keyHours)
    else:
        try:
            day = hoursString[wednesdayIndex+10:thursdayIndex]
            if day.find("los") != -1: #check if the world close or Close or closed or Closed or Closure etc.. appears
                keyHours = {'day' : "Wednesday",
                        'open' : False}
                hoursArray.append(keyHours)
            firstDigitIndex = [x.isdigit() for x in day].index(True)
            actualHours = { 'open' : day[firstDigitIndex:firstDigitIndex+5],
                            'close' : day[firstDigitIndex+6:firstDigitIndex+11]}
            keyHours = {'day' : "Wednesday",
                        'open' : True,
                        'hours' : actualHours}
            hoursArray.append(keyHours)
        except:
            keyHours = {'day' : "Wednesday",
                        'open' : False}
            hoursArray.append(keyHours)
        
    if thursdayIndex == -1:
        keyHours = {'day' : "Thursday",
                        'open' : False}
        hoursArray.append(keyHours)
    else:
        try:
            day = hoursString[thursdayIndex+9:fridayIndex]
            if day.find("los") != -1: #check if the world close or Close or closed or Closed or Closure etc.. appears
                keyHours = {'day' : "Thursday",
                        'open' : False}
                hoursArray.append(keyHours)
            firstDigitIndex = [x.isdigit() for x in day].index(True)
            actualHours = { 'open' : day[firstDigitIndex:firstDigitIndex+5],
                            'close' : day[firstDigitIndex+6:firstDigitIndex+11]}
            keyHours = {'day' : "Thursday",
                        'open' : True,
                        'hours' : actualHours}
            hoursArray.append(keyHours)
        except:
            keyHours = {'day' : "Thursday",
                        'open' : False}
            hoursArray.append(keyHours)
    if fridayIndex == -1:
        keyHours = {'day' : "Friday",
                        'open' : False}
        hoursArray.append(keyHours)
    else:
        try:
            day = hoursString[fridayIndex+7:saturdayIndex]
            if day.find("los") != -1: #check if the world close or Close or closed or Closed or Closure etc.. appears
                keyHours = {'day' : "Friday",
                        'open' : False}
                hoursArray.append(keyHours)
            firstDigitIndex = [x.isdigit() for x in day].index(True)
            actualHours = { 'open' : day[firstDigitIndex:firstDigitIndex+5],
                            'close' : day[firstDigitIndex+6:firstDigitIndex+11]}
            keyHours = {'day' : "Friday",
                        'open' : True,
                        'hours' : actualHours}
            hoursArray.append(keyHours)
        except:
            keyHours = {'day' : "Friday",
                        'open' : False}
            hoursArray.append(keyHours)
    if saturdayIndex == -1:
        keyHours = {'day' : "Saturday",
                        'open' : False}
        hoursArray.append(keyHours)
    else:
        try:
            day = hoursString[saturdayIndex+9:]
            if day.find("los") != -1: #check if the world close or Close or closed or Closed or Closure etc.. appears
                keyHours = {'day' : "Saturday",
                        'open' : False}
                hoursArray.append(keyHours)
            firstDigitIndex = [x.isdigit() for x in day].index(True)
            actualHours = { 'open' : day[firstDigitIndex:firstDigitIndex+5],
                            'close' : day[firstDigitIndex+6:firstDigitIndex+11]}
            keyHours = {'day' : "Saturday",
                        'open' : True,
                        'hours' : actualHours}
            hoursArray.append(keyHours)
        except:
            keyHours = {'day' : "Saturday",
                        'open' : False}
            hoursArray.append(keyHours)
    if sundayIndex == -1:
        keyHours = {'day' : "Sunday",
                        'open' : False}
        hoursArray.append(keyHours)
    else:
        try:
            day = hoursString[sundayIndex+7:mondayIndex]
            if day.find("los") != -1: #check if the world close or Close or closed or Closed or Closure etc.. appears
                keyHours = {'day' : "Sunday",
                        'open' : False}
                hoursArray.append(keyHours)
            firstDigitIndex = [x.isdigit() for x in day].index(True)
            actualHours = { 'open' : day[firstDigitIndex:firstDigitIndex+5],
                            'close' : day[firstDigitIndex+6:firstDigitIndex+11]}
            keyHours = {'day' : "Sunday",
                        'open' : True,
                        'hours' : actualHours}
            hoursArray.append(keyHours)
        except:
            keyHours = {'day' : "Sunday",
                        'open' : False}
            hoursArray.append(keyHours)
    return hoursArray

# Aldi's API:
## Base URL:
https://www.aldi.co.uk/api/store-finder/search
## Parameters:
* q = postcode (optional)
* address = Road, Postcode (optional)
* latitude = latidude
* longitude = longitude

## Example API call:
https://www.aldi.co.uk/api/store-finder/search?q=nr70en&address=Yarmouth+Rd%2C+Norwich+NR7+0EN&latitude=52.6266168&longitude=1.3366618
## Example response:

In [ ]:
{query: "nr70en", address: "Yarmouth Rd, Norwich NR7 0EN",…}
    address: "Yarmouth Rd, Norwich NR7 0EN"
    bounds: {n: "52.64952", e: "1.4334135999999997", s: "52.6037136", w: "1.23991"}
        e: "1.4334135999999997"
        n: "52.64952"
        s: "52.6037136"
        w: "1.23991"
    center: {lat: "52.6266168", lng: "1.3366618"}
        lat: "52.6266168"
        lng: "1.3366618"
    formattedCount: "Nearest <strong>30 stores</strong> found for <strong>Yarmouth Rd, Norwich NR7 0EN</strong>"
    pagination: {next: 1,…}
        next: 1
        pages: [{page: 0, prettyPage: 1, currentPage: true, center: {lat: "52.6266168", lng: "1.3366618"},…},…]
            0: {page: 0, prettyPage: 1, currentPage: true, center: {lat: "52.6266168", lng: "1.3366618"},…}
                address: "Yarmouth Rd, Norwich NR7 0EN"
                center: {lat: "52.6266168", lng: "1.3366618"}
                    lat: "52.6266168"
                    lng: "1.3366618"
                currentPage: true
                page: 0
                prettyPage: 1
                query: "nr70en"
            1: {page: 1, prettyPage: 2, center: {lat: "52.6266168", lng: "1.3366618"}, query: "nr70en",…}
            2: {page: 2, prettyPage: 3, center: {lat: "52.6266168", lng: "1.3366618"}, query: "nr70en",…}
            3: {page: 3, prettyPage: 4, center: {lat: "52.6266168", lng: "1.3366618"}, query: "nr70en",…}
            4: {page: 4, prettyPage: 5, center: {lat: "52.6266168", lng: "1.3366618"}, query: "nr70en",…}
            5: {page: 5, prettyPage: 6, center: {lat: "52.6266168", lng: "1.3366618"}, query: "nr70en",…}
    query: "nr70en"
    results: [,…]
        0: {code: "s-uk-E0566", name: "ALDI - 174-178 Plumstead Road", distance: "0.8 Miles", description: "",…}
            address: ["Norwich", "NR1 4JZ"]
                0: "Norwich"
                1: "NR1 4JZ"
            code: "s-uk-E0566"
            description: ""
            distance: "0.8 Miles"
            features: ["parking"]
                0: "parking"
            isFirstStore: true
            latlng: {lat: 52.6366, lng: 1.32667}
                lat: 52.6366
                lng: 1.32667
            name: "ALDI - 174-178 Plumstead Road"
            openingTimes: [{day: "Mon", hours: "08:00&nbsp;&ndash;&nbsp;22:00", closed: false, special: false},…]
                0: {day: "Mon", hours: "08:00&nbsp;&ndash;&nbsp;22:00", closed: false, special: false}
                    closed: false
                    day: "Mon"
                    hours: "08:00&nbsp;&ndash;&nbsp;22:00"
                    special: false
                1: {day: "Tue", hours: "08:00&nbsp;&ndash;&nbsp;22:00", closed: false, special: false}
                    closed: false
                    day: "Tue"
                    hours: "08:00&nbsp;&ndash;&nbsp;22:00"
                    special: false
                2: {day: "Wed", hours: "08:00&nbsp;&ndash;&nbsp;22:00", closed: false, special: false}
                    closed: false
                    day: "Wed"
                    hours: "08:00&nbsp;&ndash;&nbsp;22:00"
                    special: false
                3: {day: "Thu", hours: "08:00&nbsp;&ndash;&nbsp;22:00", closed: false, special: false}
                    closed: false
                    day: "Thu"
                    hours: "08:00&nbsp;&ndash;&nbsp;22:00"
                    special: false
                4: {day: "Fri", hours: "08:00&nbsp;&ndash;&nbsp;22:00", closed: false, special: false}
                    closed: false
                    day: "Fri"
                    hours: "08:00&nbsp;&ndash;&nbsp;22:00"
                    special: false
                5: {day: "Sat", hours: "08:00&nbsp;&ndash;&nbsp;22:00", closed: false, special: false}
                    closed: false
                    day: "Sat"
                    hours: "08:00&nbsp;&ndash;&nbsp;22:00"
                    special: false
                6: {day: "Sun", hours: "10:00&nbsp;&ndash;&nbsp;16:00", closed: false, special: false}
                    closed: false
                    day: "Sun"
                    hours: "10:00&nbsp;&ndash;&nbsp;16:00"
                    special: false
            selectedPage: "0"
        1: {code: "s-uk-E0487", name: "ALDI - 463 Sprowston Road", distance: "2 Miles", description: "",…}
        2: {code: "s-uk-E1022", name: "ALDI - Unit 5, Hall Road", distance: "2.3 Miles", description: "",…}
        3: {code: "s-uk-E1091", name: "ALDI - Unit1, 36 - 50 Drayton Road", distance: "2.6 Miles",…}
        4: {code: "s-uk-E0377", name: "ALDI - 1 Larkman Lane", distance: "4.1 Miles", description: "",…}

# Code for retreiving Aldi's opening hours data:

In [ ]:
def get_aldi_data(lat, lng):
    API_URL = "https://www.aldi.co.uk/api/store-finder/search"
    params = {  'latitude' : lat,
                'longitude' : lng
             }
    rq = requests.get(API_URL, params=params)
    if rq.status_code != 200:
        print(rq.json())
        return False
    try:
        res = rq.json()["results"]
        if len(res) == 0:
            return False
    except:
        return False
    i=0
    intermediaryDayKeys = { 'Mon' : 0, 'Tue' : 1, 'Wed' : 2, 'Thu' : 3, 'Fri' : 4, 'Sat' : 5, 'Sun' : 6}
    dayKeys = { 0 : 'Monday', 1 : 'Tuesday', 2 : 'Wednesday', 3 : 'Thursday', 4 : 'Friday', 5 : 'Saturday', 6 : 'Sunday'}
    #INSERT POTENTIAL CHECK THAT STORE MATCHES DESIRED STORENAME!! (e.g. res[0]["name"])
    #e.g.:
    # while True:
    #     if name != res[i]["name"]:
    #         i += 1
    #     else:
    #         break
    openingHours = res[i]["openingTimes"]
    hoursArray = []
    for index in range(len(openingHours)):
        key = intermediaryDayKeys[openingHours[index]['day']]  #turn key into int
        actualHours = []
        while len(hoursArray) < key:
            #Incase store is closed on a day of the week and therefore not included in the list 
            #(although this is unlikely for ALDI, this check was added for Sainsbury API since how they deal with closed days is unclear,
            #whereas it is unlikely ALDI's API would miss a day, so this check could potentially be removed if desired, however is left in for an extra layer of safety)
            closedDayHours = {  'day' : dayKeys[len(hoursArray)],
                                'open' : False  }
            hoursArray.append(closedDayHours)
        if openingHours[index]['closed'] == False:
            hoursString = html.unescape(openingHours[index]['hours'])
            while len(hoursString) > 11: #incase there are multiple time slots encoded within the string (format is unknown hence the strange code)
                timeSlot = { 'open' : hoursString[:5],
                                'close' : hoursString[8:13]}
                actualHours.append(timeSlot)
                hoursString = hoursString[13:]
                if len(hoursString) > 0:
                    try:
                        firstDigitIndex = [x.isdigit() for x in hoursString].index(True)
                        hoursString = hoursString[firstDigitIndex:]
                    except:
                        break
            keyHours = {'day' : dayKeys[key],
                'open' : True,
                'hours' : actualHours
                }
        else:
            keyHours = {'day' : dayKeys[key],
                'open' : False
                }
        
        hoursArray.append(keyHours)
    while len(hoursArray) < 7:
            #Incase store is closed on a day of the week (at the end of the list/week) and therefore not included in the list
            closedDayHours = {  'day' : dayKeys[len(hoursArray)],
                                'open' : False  }
            hoursArray.append(closedDayHours)
    return hoursArray